### Setup

In [1]:
from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from council_controller import LLMInstructController
from evaluator import BasicEvaluatorWithSource

import dotenv
import os

from string import Template
import toml
import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.INFO)

dotenv.load_dotenv()
llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

### Load auxilliary files

In [2]:
fred_system_prompt = toml.load("../agent/prompts/fred_data_specialist_prompts.toml")["system_message"]["prompt"]
print(fred_system_prompt)

You are an expert FRED data specialist. You generate or edit Python code to access the FRED economic database via the Python package fredapi.


In [3]:
fred_prompt_template = Template(toml.load("../agent/prompts/fred_data_specialist_prompts.toml")["main_prompt"]["prompt"])
print(fred_prompt_template.template)

# Task Description
Your task is to produce expert-level Python code related to the analysis of economic time series.
You will have access to data via FRED and the fredapi Python package.

# fredapi README

```markdown

# fredapi: Python API for FRED (Federal Reserve Economic Data)

`fredapi` is a Python API for the [FRED](http://research.stlouisfed.org/fred2/) data provided by the
Federal Reserve Bank of St. Louis. `fredapi` provides a wrapper in python to the 
[FRED web service](http://api.stlouisfed.org/docs/fred/), and also provides several convenient methods
for parsing and analyzing point-in-time data (i.e. historic data revisions) from [ALFRED](http://research.stlouisfed.org/tips/alfred/)

`fredapi` makes use of `pandas` and returns data to you in a `pandas` `Series` or `DataFrame`

## Basic Usage

First you need an API key, you can [apply for one](http://api.stlouisfed.org/api_key.html) for free on the FRED website.
Once you have your API key, you can set it in one of three ways

In [4]:
code_header = toml.load("./prompts/description.toml")['code_header']['code']
print(code_header)

import pandas as pd
from fredapi import Fred
import plotly.io as pio
pio.renderers.default = "firefox"

fred = Fred(api_key=os.getenv("FRED_API_KEY"))



In [5]:
code_editor_system_prompt = toml.load("./prompts/code_editor_prompt.toml")['system_message']['prompt']
print(code_editor_system_prompt)

You are an AI data scientist specialized in economic time series data. You have expert-level skills in data analytics using Python, Pandas, scikit-learn, and scipy. You use plotly for data visualization.


In [6]:
code_editor_prompt_template = Template(toml.load("./prompts/code_editor_prompt.toml")['main_prompt']['prompt'])
print(code_editor_prompt_template.template)

# TASK DESCRIPTION
Your task is to write or edit expert-level Python code related to the analysis of economic time series.

## EXISTING PYTHON CODE
$existing_code

## REQUIRED CODE HEADER
Always make sure the SOLUTION begins with the following code snippet:
$code_header

# INSTRUCTIONS
- If possible, edit the EXISTING PYTHON CODE to solve the Task.
- Otherwise, write a new script to solve the Task.
- Begin with comments that explain your step-by-step approach to solving the problem.
- Place all code in a single code block, formatted as ```python {code} ```
- All available and required data are accessible in `data_sources`. Use only these dataframes to solve the task.
- Follow PEP 8 style guides, including a max line length of 79 characters.
- Always include print statements to interact with the user, including when you need more details.
- Always include a print statement at the end of the script to summarize what you've done for the user.
- Never use the input function to request inpu

In [7]:
code_correction_system_prompt = toml.load("./prompts/code_correction_prompt.toml")['system_message']['prompt']
print(code_correction_system_prompt)

You are an expert Python code reviewer.


In [8]:
code_correction_prompt_template = Template(toml.load("./prompts/code_correction_prompt.toml")['main_prompt']['prompt'])
print(code_correction_prompt_template.template)

# Task Description
Your task is to correct errors in Python code.
After reviewing the PYTHON CODE, ERROR MESSAGE, and TASK, you will generate corrected Python code.

## PYTHON CODE
$code

## ERROR MESSAGE
$error_message

## REQUIRED CODE HEADER
Always begin your SOLUTION with the following code snippet:
$code_header

## INSTRUCTIONS
- Edit the ## Python Code to solve the Task.
- Include comments that explain your step-by-step approach to resolving the problem.
- Place all code in a single code block, formatted as ```python {code} ```
- All available and required data are accessible in `data_sources`. Use only these dataframes to solve the task.
- Follow PEP 8 style guides, including a max line length of 79 characters.
- Always include print statements to interact with the user, including when you need more details.
- Always include a print statement at the end of the script to summarize what you've done for the user.
- Never use the input function to request input from the user. Instea

### Build Skills, Chains, and Agents

#### Skills

In [9]:
from skills import FredDataSpecialist, PythonCodeEditorSkill, ParsePythonSkill, PythonExecutionSkill, GeneralSkill

"""
FRED Data Specialist
"""
fred_data_specialist = FredDataSpecialist(
    llm,
    system_prompt=fred_system_prompt,
    main_prompt_template=fred_prompt_template,
    code_header=code_header,
)

"""
Specialized Python code editing skill for daily securities price analysis.
"""
code_editing_skill = PythonCodeEditorSkill(
    llm,
    system_prompt=code_editor_system_prompt,
    editor_prompt_template=code_editor_prompt_template,
    code_header=code_header
)

"""
Validate/parse Python code block - could easily be generalized to regex pattern matching skill.
"""
parse_python_skill = ParsePythonSkill()

"""
Execute Python code locally in host environment - UNSAFE.
"""
python_execution_skill = PythonExecutionSkill(
    llm,
    code_correction_system_prompt,
    code_correction_prompt_template,
    code_header,
    python_bin_dir=os.environ['PYTHON_BIN_DIR']
)

"""
General Skill
"""
general_skill = GeneralSkill(llm, system_prompt="""You are a friendly, helpful assistant. 
            Generate a brief response accoring to the provided instruction; 2 sentences at most.""")

#### Chains

In [10]:
fred_data_specialist_chain = Chain(
    name="fred_data_specialist",
    description="Identify and access economic datasets from the FRED database. Use this chain when you need to generate or edit code for accessing or downloading data from FRED.",
    runners=[fred_data_specialist, parse_python_skill]
)

code_editing_and_execution_chain = Chain(
    name="data_analysis_code_editing_and_execution",
    description="Generate/edit and execute existing Python code for data analytics and visualization. Use this chain if the user wants to generate new code or edit existing code related to data analysis.",
    runners=[
        code_editing_skill,
        parse_python_skill,
        python_execution_skill,
    ],
)

code_editing_chain = Chain(
    name="data_analysis_code_editing",
    description="Generate/edit (but do not execute) Python code for data analytics and visualization. Use this chain if the user wants to generate new code or edit existing code related to data analysis.",
    runners=[
        code_editing_skill,
        parse_python_skill,
    ],
)

code_execution_chain = Chain(
    name="code_execution_and_correction",
    description="Execute (but do not edit) existing Python code for data analytics and visualization. Use this chain if the user wants to run existing code.",
    runners=[
        parse_python_skill,
        python_execution_skill,
    ],
)

general_chain = Chain(
    name="general",
    description="Answer general questions without the use of any specialized skills. Use this when the user needs the answer to a question that doesn't require any coding.",
    runners=[general_skill],
)

#### Agent

In [11]:
controller = LLMInstructController(
    llm=llm, 
    hints=[],
    top_k_execution_plan=1
)
evaluator = BasicEvaluatorWithSource()

agent = Agent(
    controller=controller, 
    chains=[fred_data_specialist_chain, code_editing_chain, code_editing_and_execution_chain, code_execution_chain, general_chain], 
    evaluator=evaluator
)

agent.controller._state["code"] = None

### Run the Agent

In [12]:
def interact(agent, context, message):
    print(f"User Message: {message}")
    context.chatHistory.add_user_message(message)
    result = agent.execute(context=context, budget=Budget(180))
    print(result.best_message.message)
    print()

chat_history = ChatHistory()
context = AgentContext(chat_history=chat_history)
message = "I want to do an analysis of economic time series that are either directly or indirectly related to food consumer price index."
interact(agent, context, message)

[2023-08-08 13:59:32-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-08-08 13:59:32-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"


User Message: I want to do an analysis of economic time series that are either directly or indirectly related to food consumer price index.


[2023-08-08 13:59:35-0400 INFO MainThread council:get_plan:130] Controller Message: fred_data_specialist;8;Please identify and access economic datasets from the FRED database that are directly or indirectly related to the food consumer price index.
[2023-08-08 13:59:35-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="fred_data_specialist" execution_unit="fred_data_specialist;8"
[2023-08-08 13:59:35-0400 INFO chain_fred_data_specialist_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="FredDataSpecialist"
[2023-08-08 13:59:49-0400 INFO chain_fred_data_specialist_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="FredDataSpecialist" skill_message="I've generated code for you and placed it in the 'data' field."
[2023-08-08 13:59:49-0400 INFO chain_fred_data_specialist_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="ParsePythonSkill"
[20

Parsing succeeded.



In [13]:
print(agent.controller._state["code"])

import pandas as pd
from fredapi import Fred
import os
import plotly.io as pio
pio.renderers.default = "firefox"

fred = Fred(api_key=os.getenv("FRED_API_KEY"))

# Search for data series related to the food consumer price index
search_results = fred.search('food consumer price index')
print(search_results.T)

# Get the series id for the first result
series_id = search_results.index[0]

# Get the data for this series
data = fred.get_series(series_id)

# Print the data
print(data)

# Summary
print(f"Accessed the '{series_id}' series from the FRED database, which is related to the food consumer price index.")


In [14]:
message = "can you please run it?"
interact(agent, context, message)

[2023-08-08 13:59:49-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-08-08 13:59:49-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="2"


User Message: can you please run it?


[2023-08-08 13:59:54-0400 INFO MainThread council:get_plan:130] Controller Message: code_execution_and_correction;9;Please execute the provided Python code which accesses data from the FRED database and performs an analysis on the food consumer price index.
[2023-08-08 13:59:54-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="code_execution_and_correction" execution_unit="code_execution_and_correction;9"
[2023-08-08 13:59:54-0400 INFO chain_code_execution_and_correction_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="ParsePythonSkill"
[2023-08-08 13:59:54-0400 INFO chain_code_execution_and_correction_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="ParsePythonSkill" skill_message="Parsing succeded."
[2023-08-08 13:59:54-0400 INFO chain_code_execution_and_correction_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="PythonExecutionS

Starting execution...


[2023-08-08 13:59:57-0400 INFO chain_code_execution_and_correction_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="PythonExecutionSkill" skill_message="series id                                                           CPIAUCSL  ...                                 00XFUNEZ19M086NEST
id                                                                  CPIAUCSL  ...                                 00XFUNEZ19M086NEST
realtime_start                                           2023-08-08 00:00:00  ...                                2023-08-08 00:00:00
realtime_end                                             2023-08-08 00:00:00  ...                                2023-08-08 00:00:00
title                      Consumer Price Index for All Urban Consumers: ...  ...  Harmonized Index of Consumer Prices: Overall I...
observation_start                                        1947-01-01 00:00:00  ...                                2000-12-01 00:00:00
observation_e

series id                                                           CPIAUCSL  ...                                 00XFUNEZ19M086NEST
id                                                                  CPIAUCSL  ...                                 00XFUNEZ19M086NEST
realtime_start                                           2023-08-08 00:00:00  ...                                2023-08-08 00:00:00
realtime_end                                             2023-08-08 00:00:00  ...                                2023-08-08 00:00:00
title                      Consumer Price Index for All Urban Consumers: ...  ...  Harmonized Index of Consumer Prices: Overall I...
observation_start                                        1947-01-01 00:00:00  ...                                2000-12-01 00:00:00
observation_end                                          2023-06-01 00:00:00  ...                                2023-06-01 00:00:00
frequency                                                            

In [15]:
message = "can we plot the top 10?"
interact(agent, context, message)

[2023-08-08 13:59:57-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-08-08 13:59:57-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="3"


User Message: can we plot the top 10?


[2023-08-08 14:00:05-0400 INFO MainThread council:get_plan:130] Controller Message: data_analysis_code_editing_and_execution;8;Please generate and execute Python code to plot the top 10 data series related to the food consumer price index from the previously accessed FRED data.
[2023-08-08 14:00:05-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="data_analysis_code_editing_and_execution" execution_unit="data_analysis_code_editing_and_execution;8"
[2023-08-08 14:00:05-0400 INFO chain_data_analysis_code_editing_and_execution_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="PythonCodeEditorSkill"
[2023-08-08 14:00:55-0400 INFO chain_data_analysis_code_editing_and_execution_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="PythonCodeEditorSkill" skill_message="I've edited code for you and placed the result in the 'data' field."
[2023-08-08 14:00:55-0400 INFO chain_data_ana

Starting execution...


[2023-08-08 14:01:04-0400 INFO chain_data_analysis_code_editing_and_execution_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="PythonExecutionSkill" skill_message="Plotted the top 10 data series related to the food consumer price index from the FRED database."
[2023-08-08 14:01:04-0400 INFO MainThread council.agents.agent:execute:76] message="chain execution ended" chain="data_analysis_code_editing_and_execution" execution_unit="data_analysis_code_editing_and_execution;8"
[2023-08-08 14:01:04-0400 INFO MainThread council.agents.agent:execute:88] message="agent execution ended"


Plotted the top 10 data series related to the food consumer price index from the FRED database.



In [16]:
message = "scale the data before plotting"
interact(agent, context, message)

[2023-08-08 14:01:04-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-08-08 14:01:04-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="4"


User Message: scale the data before plotting


[2023-08-08 14:01:12-0400 INFO MainThread council:get_plan:130] Controller Message: data_analysis_code_editing_and_execution;9;The user wants to scale the data before plotting. Please edit the existing code to include a data scaling step before the plotting step. You can use a suitable scaling method like MinMaxScaler or StandardScaler from sklearn.preprocessing. After scaling, proceed with the plotting as before.
[2023-08-08 14:01:12-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="data_analysis_code_editing_and_execution" execution_unit="data_analysis_code_editing_and_execution;9"
[2023-08-08 14:01:12-0400 INFO chain_data_analysis_code_editing_and_execution_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="PythonCodeEditorSkill"
[2023-08-08 14:01:30-0400 INFO MainThread council.agents.agent:execute:88] message="agent execution ended"


KeyboardInterrupt: 

[2023-08-08 14:01:59-0400 INFO chain_data_analysis_code_editing_and_execution_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="PythonCodeEditorSkill" skill_message="I've edited code for you and placed the result in the 'data' field."
